# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
vacation=("cities_updated.csv")
weather_data=pd.read_csv(vacation)
weather_data


,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Hermanus,-34.4187,19.2345,71.01,73,0,5.99,ZA,1619122018
1,Bluff,-46.6000,168.3333,45.00,97,100,3.00,NZ,1619121897
2,Ugoofaaru,5.6667,73.0000,82.06,78,20,9.71,MV,1619122018
3,Vostok,46.4856,135.8833,43.41,55,100,3.94,RU,1619122019
4,Forsytheganj,52.6333,29.7333,45.55,90,94,4.09,BY,1619121931
...,...,...,...,...,...,...,...,...,...
568,Tamuín,21.9833,-98.7500,93.99,34,57,9.57,MX,1619122216
569,Berdigestyakh,62.0989,126.6992,9.84,97,99,4.68,RU,1619122217
570,Ust'-Omchug,61.1500,149.6333,22.93,95,100,3.47,RU,1619122217
571,Pavlodar,52.3000,76.9500,19.40,79,0,6.71,KZ,1619122217


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
locations

Lat    0
Lng    0
dtype: int64

In [60]:
gmaps.configure(api_key=g_key)

locations=weather_data[['Lat', 'Lng']]
# locations.columns=['latitude', 'longitude']
weight=weather_data['Humidity']

m=gmaps.Map()
# heatmap_layer=gmaps.Heatmap(locations)
heatmap_layer=gmaps.heatmap_layer(locations, weights=weight)
m.add_layer(heatmap_layer)



### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [46]:
perfect_weather=weather_data[(weather_data['Max Temp']>=70) & (weather_data['Max Temp']<=80)]
perfect_weather=perfect_weather[perfect_weather['Wind Speed']<=10]
perfect_weather=perfect_weather[perfect_weather['Clouds']<=10]
perfect_weather=perfect_weather[perfect_weather['Humidity']<=60]
perfect_weathers=perfect_weather.dropna()
perfect_weathers

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
26,New Iberia,30.0035,-91.8187,70.90,37,1,4.61,US,1619121772
74,Chiang Klang,19.2938,100.8617,79.00,46,6,1.48,TH,1619122041
278,Nushki,29.5542,66.0215,74.61,12,0,5.95,PK,1619122064
309,Laguna,38.4210,-121.4238,70.25,45,1,9.22,US,1619122122
363,Bela,25.9333,81.9833,79.16,26,0,7.94,IN,1619122140
437,Marzuq,14.4000,46.4667,71.24,47,0,4.54,YE,1619122167
454,Karratha,-20.7377,116.8463,76.06,26,0,9.93,AU,1619122173
488,Saint George,37.1041,-113.5841,73.67,18,1,3.44,US,1619122185
497,Umuarama,-23.7664,-53.3250,78.87,41,0,5.66,BR,1619122188
509,Mendota,36.7536,-120.3816,73.17,40,1,5.75,US,1619122192


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [55]:
perfect_weathers['Hotel Name']=""
hotel_df=perfect_weathers
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
26,New Iberia,30.0035,-91.8187,70.90,37,1,4.61,US,1619121772,
74,Chiang Klang,19.2938,100.8617,79.00,46,6,1.48,TH,1619122041,
278,Nushki,29.5542,66.0215,74.61,12,0,5.95,PK,1619122064,
309,Laguna,38.4210,-121.4238,70.25,45,1,9.22,US,1619122122,
363,Bela,25.9333,81.9833,79.16,26,0,7.94,IN,1619122140,
437,Marzuq,14.4000,46.4667,71.24,47,0,4.54,YE,1619122167,
454,Karratha,-20.7377,116.8463,76.06,26,0,9.93,AU,1619122173,
488,Saint George,37.1041,-113.5841,73.67,18,1,3.44,US,1619122185,
497,Umuarama,-23.7664,-53.3250,78.87,41,0,5.66,BR,1619122188,
509,Mendota,36.7536,-120.3816,73.17,40,1,5.75,US,1619122192,


In [56]:
for index, row in hotel_df.iterrows():
    try:
        url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params={"keyword":"hotel", "radius": 5000, "key":g_key}
        lat=row['Lat']
        lng=row["Lng"]
        params['location']=f"{lat},{lng}"
        hotel_info=requests.get(url,params=params).json()
        hotel_df.loc[index, "Hotel Name"]=hotel_info["results"][0]['name']
    except IndexError:
        hotel_df.loc[index,"Hotel Name"]="NA"
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
26,New Iberia,30.0035,-91.8187,70.90,37,1,4.61,US,1619121772,Hampton Inn & Suites New Iberia Avery Island
74,Chiang Klang,19.2938,100.8617,79.00,46,6,1.48,TH,1619122041,Sangthong Resort
278,Nushki,29.5542,66.0215,74.61,12,0,5.95,PK,1619122064,Nabi Bux Hotel
309,Laguna,38.4210,-121.4238,70.25,45,1,9.22,US,1619122122,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
363,Bela,25.9333,81.9833,79.16,26,0,7.94,IN,1619122140,SPOT ON 46397 Leela Palace
437,Marzuq,14.4000,46.4667,71.24,47,0,4.54,YE,1619122167,NA
454,Karratha,-20.7377,116.8463,76.06,26,0,9.93,AU,1619122173,Latitude20 The Dunes Apartments
488,Saint George,37.1041,-113.5841,73.67,18,1,3.44,US,1619122185,Red Lion Hotel & Conference Center St. George
497,Umuarama,-23.7664,-53.3250,78.87,41,0,5.66,BR,1619122188,Hotel Raf
509,Mendota,36.7536,-120.3816,73.17,40,1,5.75,US,1619122192,NA


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# Add marker layer ontop of heat map
fig=gmaps.figure()

markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))